In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_KEY=os.environ["OPENAI_API_KEY"]

In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('../data/RaptorContract.pdf')
pages = loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_KEY)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
import weaviate
from langchain.vectorstores import Weaviate

client = weaviate.Client(
    url="http://localhost:8080",
    additional_headers={"X-OpenAI-Api-Key": OPENAI_KEY},
    startup_period=10
)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [13]:
# define input structure
client.schema.delete_all()
client.schema.get()
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-openai",
            "moduleConfig": {"text2vec-openai": {"model": "ada", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-openai": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}

client.schema.create(schema)

vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])



In [14]:
# load text into the vectorstore
text_meta_pair = [(doc.page_content, doc.metadata) for doc in docs]
texts, meta = list(zip(*text_meta_pair))
vectorstore.add_texts(texts, meta)

['3c59df83-706c-4665-9cc7-7bfdc89e2976',
 'f998ab68-9875-43b8-8680-4b068dbf5604',
 'a3588925-c9a8-421c-9082-7ec5bf654467',
 '15cfb2a0-d91e-43fc-861a-bb22040f6426',
 'd976cedb-c8c0-44d7-90b5-bd75b7303b02',
 'b0acfbc4-2188-4929-94fb-6546919b1904',
 '75a09ed9-524c-4a43-9624-17610e430ee6',
 'bd9f50ed-1b09-404c-b823-21c67636cabd',
 '40e24dcc-332e-4b3b-b82c-efffed6001d4',
 '407c4872-0cc5-419e-86b4-1ab336f0b997',
 'f6c9313c-5bd8-4dfc-abc8-2819824df230',
 '40dc5035-a2ea-487f-89ab-e2b738e2ac88',
 '303ccdc4-6bf5-41cc-85d4-3cbd68f90204',
 '3c525f05-9163-4d22-b57d-34b4640bec80',
 'c2d8f06e-0024-4271-a8d4-fe7aad6ca441',
 '2b444054-24a8-4e79-8ffc-ab3b432eecc4',
 '6554c9ac-3b1d-428f-9e42-5065c63f050d',
 '808ecfa9-ab2f-4a46-83e4-511756f085f8',
 '789725b4-f35f-4c18-8b0c-bbd54504b8c7',
 '97b9ab6c-51dc-4404-81f6-2734f339a35d',
 'b124a91d-fbdc-4d76-a28e-ca787b7afaef',
 '9fd4946b-d596-4cb9-99c2-910b1c0bf909',
 '20fe71e8-6141-4766-8bbc-4e9f6c483a42',
 'e71534c6-2314-4ebb-bc4b-c4bf198e35e4',
 '11ea57dd-2997-

In [15]:
query = "Is there a non-compete obligation to the Advisor?"

# retrieve text related to the query
docs = vectorstore.similarity_search(query, k=3)

In [33]:
query2 = "Are there any conditions to the closing?"

docs2 = vectorstore.similarity_search(query2, k=10)

In [34]:
chain.run(input_documents=docs2, question=query2)


" Yes, there are conditions to the closing. The closing will take place on the Closing Date, and the Buyer will pay the Closing Cash Consideration, Warrant Closing Cash Consideration, and Closing Vested Options Cash Consideration. The Sellers will also deliver certain documents and the Buyer will rely on the appointment and authority of the Sellers' Representative."

In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# define chain
chain = load_qa_chain(
    OpenAI(openai_api_key = OPENAI_KEY,temperature=0),
    chain_type="stuff")

# create answer
chain.run(input_documents=docs, question=query)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\nNo, there is no non-compete obligation to the Advisor.'